<a href="https://colab.research.google.com/github/LucasLin2023/Cyberbullying/blob/main/20240809__%E5%AE%9A%E9%A3%8E%E6%B3%A2%E4%BA%8B%E4%BB%B6_%E9%9A%8F%E6%89%8B%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


把评论区数据从github 拿过来~~

In [ ]:
from google.colab import userdata
my_api = userdata.get('my_openai_api') # you have to store your own openai api key in secret key to use llm.

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/LucasLin2023/Cyberbullying.git
%cd Cyberbullying
!ls

Cloning into 'Cyberbullying'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 209 (delta 98), reused 62 (delta 53), pack-reused 75 (from 1)
Receiving objects: 100% (209/209), 10.77 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/content/Cyberbullying/Cyberbullying
20240806_1742_陈丽君偷歌话题相关微博.csv  README.md	      topic_trend_with_explanation.png
chinese.msyh.ttf			  response_trend.png  userinfo
comments				  stopwords-zh.txt    相关文件描述.txt
freq_poster.png				  topics


In [ ]:
import pandas as pd
import numpy as np
import warnings
import glob
import os

warnings.filterwarnings('ignore')

In [ ]:
glob.glob("./topics/*.csv")

['./topics/20240808_2151_陈丽君回应周深粉丝话题相关微博.csv',
 './topics/20240808_2151_相关用户信息.csv',
 './topics/20240808_2151_陈丽君回复周深粉丝话题相关微博.csv',
 './topics/20240808_2151_陈丽君偷歌话题相关微博.csv']

In [ ]:
glob.glob("./comments/*.csv")

['./comments/20240807_1751_陈丽君评论区热评.csv',
 './comments/20240809_1159_陈丽君评论区热评.csv',
 './comments/20240807_1137_陈丽君评论区热评.csv']

In [ ]:
df_comment = pd.read_csv('./comments/20240809_1159_陈丽君评论区热评.csv')
df_user = pd.read_csv('./topics/20240808_2151_相关用户信息.csv')
df_tg = pd.read_csv('./topics/20240808_2151_陈丽君偷歌话题相关微博.csv')
df_user = df_user[['微博发布者id','微博简介', '认证状态', '认证理由']].rename(columns={'微博发布者id':'评论者id'})

## 陈丽君老师微博评论区小分析

In [ ]:
df_comment.head()

,评论id,评论者id,评论时间,评论者姓名,评论者IP归属地,评论者粉籍,评论内容,fan_cnt,like_cnt
0,5063834022707959,6761369485,2024-08-05 02:19:35,大土豆你个破忒头_,来自福建,loyal_fans,宝宝唱的真好 快点休息,1.1万,12666
1,5063833976047058,7444504753,2024-08-05 02:19:24,不想穿校裤,来自湖北,loyal_fans,宝宝你超棒的！,2123,10384
2,5063833952191436,7767636727,2024-08-05 02:19:18,搅碎一池星月--,来自陕西,loyal_fans,宝宝。你是最棒的,77,9513
3,5063834012222504,7007259210,2024-08-05 02:19:33,漠然-淡然,来自江苏,loyal_fans,啊仙君你来了早点休息辛苦了,58,9072
4,5063834006195073,7742534781,2024-08-05 02:19:31,阿Mei的好多天_,来自陕西,loyal_fans,宝宝你特别棒！！超级美,64,8317


过滤无粉丝tag的用户


In [ ]:
df_non_fan = df_comment[df_comment['评论者粉籍']!='loyal_fans']

In [ ]:
print(f"{len(df_non_fan)} 条评论来自无铁粉标签的用户，其余{len(df_comment)-len(df_non_fan)}条来自明确来自粉丝")

410 条评论来自无铁粉标签的用户，其余1243条来自明确来自粉丝


In [ ]:
fan_kw = ['宝宝','君君','君宝','小俊'] #你们叫的好亲昵哦
pattern = '|'.join(fan_kw)
filtered_df = df_non_fan[~df_non_fan['评论内容'].str.contains(pattern, na=False)]
print(f"去掉含有粉丝常用关键词后还剩 {len(filtered_df)} 条评论作为分析使用 ")

去掉含有粉丝常用关键词后还剩 352 条评论作为分析使用 


统计评论区文案的出现频率，并由高到低排列


In [ ]:
filtered_df.groupby('评论内容').agg({'评论者姓名':'count'})\
.reset_index().rename(columns={'评论者姓名':'相同文案出现频率'}).sort_values('相同文案出现频率',ascending=False)

,评论内容,相同文案出现频率
29,不要再狡辩了📢将《光亮》里🔥出圈的京剧戏腔改名《定风波》，模糊原唱并营销是越剧选段的是你，引...,22
210,这两年您在其他平台多次演唱百万收藏大爆曲《光亮》戏腔不标明原曲甚至擅自改名成《定风波》，很难...,21
233,陈女士营销宣传将《光亮》里🔥出圈的京剧戏腔改名《定风波》，利用这个模糊原唱营销是越剧选段，企...,20
205,这两年您在其他平台多次演唱百万收藏大爆曲《光亮》且擅自改名成《定风波》误导大众以为是传统越剧...,18
208,这两年您在其他平台多次演唱百万收藏大爆曲《光亮》戏腔不标明原曲甚至擅自改名成《定风波》，很难...,13
...,...,...
86,君，咱能做的也都做了，剩下的都别往心里去，你已经很优秀了,1
87,听我一句劝，陈丽君粉丝现在最重要的是小升初,1
88,和主办方确认过版权问题???确认在哪呢我请问，就不能不唱吗？这么喜欢定风波这词，自己哼一段读...,1
89,哈哈哈哈哈偷歌贼,1


发现相似文案，集中在频率最高的几条， 使用tf-idf算法取相似度并将相似文案做统合，设定similarity score 为0.6（可调整）


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Calculate TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer().fit_transform(filtered_df['评论内容'])
tfidf_matrix = tfidf_vectorizer.toarray()

# Calculate cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

# Threshold for similarity
threshold = 0.6

# Group similar texts and standardize them
def group_similar_texts(df, col, cosine_sim, threshold):
    # Create a list to store the group id for each text
    groups = [-1] * len(df)
    group_id = 0

    for i in range(len(df)):
        if groups[i] == -1:  # If the text is not yet assigned to a group
            groups[i] = group_id
            for j in range(i + 1, len(df)):
                if cosine_sim[i][j] > threshold:
                    groups[j] = group_id
            group_id += 1

    # Assign the standardized text to each group
    standardized_texts = {}
    for group in set(groups):
        group_indices = [i for i, g in enumerate(groups) if g == group]
        # Use the first text in the group as the standardized text
        standardized_text = df[col].iloc[group_indices[0]]
        standardized_texts[group] = standardized_text

    # Replace text in the DataFrame
    df['standardized_content'] = [standardized_texts[groups[i]] for i in range(len(df))]
    return df

# Apply the function
filtered_df = group_similar_texts(filtered_df,'评论内容', cosine_sim, threshold)\
.rename(columns={'standardized_content':'评论内容_std'})

将统合后的文案再次进行频率统计，并由高到低排列，可以发现top 4 文案可能存在刷版现象。

In [ ]:
df_freq = filtered_df.groupby('评论内容_std')\
.agg(相同文案出现频率 = ('评论者姓名','count'),
     总获赞数 = ('like_cnt','sum'),
     平均获赞数 = ('like_cnt',lambda x: round(x.mean(), 0)))\
.reset_index().sort_values('相同文案出现频率',ascending=False)

df_freq

,评论内容_std,相同文案出现频率,总获赞数,平均获赞数
185,这两年您在其他平台多次演唱百万收藏大爆曲《光亮》戏腔不标明原曲甚至擅自改名成《定风波》，很难...,70,18632,266.0
199,陈女士营销宣传将《光亮》里🔥出圈的京剧戏腔改名《定风波》，利用这个模糊原唱营销是越剧选段，企...,26,3261,125.0
19,不要再狡辩了📢将《光亮》里🔥出圈的京剧戏腔改名《定风波》，模糊原唱并营销是越剧选段的是你，引...,26,2863,110.0
164,终于等来了迟来两年的道歉文艺工作者的底线是尊重原创和版权，引导粉丝向真善美，而不是假恶丑既然...,7,4982,712.0
105,总结事件脉络:【陈丽君营销宣传将《光亮》里🔥出圈的京剧戏腔改名《定风波》，利用这个模糊原唱营...,3,249,83.0
...,...,...,...,...
73,君，咱能做的也都做了，剩下的都别往心里去，你已经很优秀了,1,1,1.0
74,听我一句劝，陈丽君粉丝现在最重要的是小升初,1,0,0.0
75,和主办方确认过版权问题???确认在哪呢我请问，就不能不唱吗？这么喜欢定风波这词，自己哼一段读...,1,1,1.0
76,哈哈哈哈哈偷歌贼,1,0,0.0


#### 小结论：经过统合后前四高频率文案可能来自“屠评论区”计划


让我们再来看看是谁在发这些定制文案~~

In [ ]:
df_comment.columns

Index(['评论id', '评论者id', '评论时间', '评论者姓名', '评论者IP归属地', '评论者粉籍', '评论内容',
       'fan_cnt', 'like_cnt'],
      dtype='object')

In [ ]:
df_user.columns

Index(['评论者id', '微博简介', '认证状态', '认证理由'], dtype='object')

计算非粉丝评论中明确为周深大粉的数量

In [ ]:
filtered_df_new = filtered_df.merge(df_user,on='评论者id',how='left')
zs_related_user_cnt_0 = filtered_df_new[((filtered_df_new['认证理由'].str.contains('周深|啾星')) |\
             (filtered_df_new['微博简介'].str.contains('周深|啾星')))]['评论者id'].nunique()
print(f"在非粉丝评论中共计 {zs_related_user_cnt_0} 名用户系周深大粉")

在非粉丝评论中共计 57 名用户系周深大粉


计算使用定制文案的周深大粉的数量

In [ ]:
## select top 4 most frequent comments
freq_content_lst = list(df_freq['评论内容_std'][:4])

In [ ]:
# merge with user profile
filtered_df_ = filtered_df[filtered_df['评论内容_std'].isin(freq_content_lst)].merge(df_user,on='评论者id',how='left')

In [ ]:
print(f"总共 {len(filtered_df_)} 名用户发表定制文案")

总共 129 名用户发表定制文案


简单看看这些发表定制文案的用户是谁，选微博简介跟认证理由两列检视。


In [ ]:
filtered_df_[filtered_df_['认证状态']==True][['微博简介','认证理由']].drop_duplicates()

,微博简介,认证理由
1,周深永远是我最可爱的宝宝！我以渺小爱你🥰！ 详细请看置顶哦,微博原创视频博主
2,Zstation_forCharlie,超话粉丝大咖（周深超话）
4,反深代词神专驾到，糊坛通通闪开👋,超话粉丝大咖（周深超话）
5,我只在乎你 镇魂,超话粉丝大咖（周深超话）
6,固执爱一朵小深花🍑 唯💞歌坛巨星脏脏包（淡网版）,娱乐博主 超话粉丝大咖（周深超话） 微博剪辑视频博主
13,镇魂,超话粉丝大咖（周深超话）
14,事在人为 和光同尘,超话粉丝大咖（周深超话） 微博剪辑视频博主
17,小狗诗集 里面写的是怎么爱小猫🤲 是恶犬,超话粉丝大咖（周深超话）
18,万物皆有裂痕，那是光照进来都地方，我燃这一生追光逐焰。,超话粉丝大咖（周深超话） 微博原创视频博主
22,我唯一的猫猫神——周深,超话粉丝大咖（周深超话） 微博原创视频博主


选取关键词“周深”or“啾星”，统计有多少是周深认证大粉

In [ ]:
zs_related_user_cnt = filtered_df_[((filtered_df_['认证理由'].str.contains('周深|啾星')) |\
             (filtered_df_['微博简介'].str.contains('周深|啾星')))]['评论者id'].nunique()
print(f"共计 {zs_related_user_cnt} 名用户系周深大粉")

共计 37 名用户系周深大粉


In [ ]:
total_likes_from_zs_fan = filtered_df_[((filtered_df_['认证理由'].str.contains('周深|啾星')) |\
             (filtered_df_['微博简介'].str.contains('周深|啾星')))]['like_cnt'].sum()
print(f"使用固定文案的周深大粉的评论共计或者点赞量为 {total_likes_from_zs_fan}")

使用固定文案的周深大粉的评论共计或者点赞量为 18228


In [ ]:
total_likes_from_zs_fan_2 = filtered_df_new[((filtered_df_new['认证理由'].str.contains('周深|啾星')) |\
             (filtered_df_new['微博简介'].str.contains('周深|啾星')))]['like_cnt'].sum()
print(f"周深大粉的评论共计或者点赞量为 {total_likes_from_zs_fan_2}")

周深大粉的评论共计或者点赞量为 36407


##偷歌话题的简单小分析

In [ ]:
df_tg['微博发布时间'] = pd.to_datetime(df_tg['微博发布时间'])

In [ ]:
first_mblog = df_tg['微博发布时间'].max().strftime('%Y-%m-%d, %H:%M:%S')
last_mblog = df_tg['微博发布时间'].min().strftime('%Y-%m-%d, %H:%M:%S')
print(f'首条 #陈丽君 偷歌 话题下微博发布于 {first_mblog} \n该文件采集到最近一条该话题下微博发布于 {last_mblog}')

首条 #陈丽君 偷歌 话题下微博发布于 2024-08-08, 19:30:52 
该文件采集到最近一条该话题下微博发布于 2024-08-04, 21:17:14


来看看一共有多少条 陈丽君偷歌 相关微博

In [ ]:
print(f"截止2024年8月8日，据不完全统计，\n共采集到来自{df_tg['微博发布者id'].nunique()}名用户发表的 陈丽君偷歌相关话题微博 {df_tg['微博id'].nunique()}条，\
\n合计转发 {df_tg['repost_cnt'].sum()}次，\
\n合计评论 {df_tg['comment_cnt'].sum()}条，\
\n合计点赞 {df_tg['like_cnt'].sum()}次。")

print('--------------------------')
df_tg_verified = df_tg[df_tg['认证状态']==True]
print(f"其中来自于{df_tg_verified['微博发布者id'].nunique()}名认证用户微博 {df_tg_verified['微博id'].nunique()}条\
\n合计转发 {df_tg_verified['repost_cnt'].sum()}次，\
\n合计评论 {df_tg_verified['comment_cnt'].sum()}条，\
\n合计点赞 {df_tg_verified['like_cnt'].sum()}次。")
print('--------------------------')
df_tg_zs = df_tg_verified[((df_tg_verified['认证理由'].str.contains('周深|啾星')) |\
             (df_tg_verified['微博简介'].str.contains('周深|啾星')))]
print(f"其中来自于{df_tg_zs['微博发布者id'].nunique()}名带有周深或者啾星相关的认证用户微博 {df_tg_zs['微博id'].nunique()}条，\
\n合计转发 {df_tg_zs['repost_cnt'].sum()}次，占所有传播次数中的{round(df_tg_zs['repost_cnt'].sum()/df_tg['repost_cnt'].sum()*100,2)}%\
\n合计评论 {df_tg_zs['comment_cnt'].sum()}条，\
\n合计点赞 {df_tg_zs['like_cnt'].sum()}次。")

截止2024年8月8日，据不完全统计，
共采集到来自1679名用户发表的 陈丽君偷歌相关话题微博 3091条，
合计转发 74371次，
合计评论 27208条，
合计点赞 328501次。
--------------------------
其中来自于317名认证用户微博 668条
合计转发 71618次，
合计评论 21607条，
合计点赞 290142次。
--------------------------
其中来自于272名带有周深或者啾星相关的认证用户微博 590条，
合计转发 52803次，占所有传播次数中的71.0%
合计评论 15106条，
合计点赞 210081次。


调用chatgpt API 对微博内容进行简单语义分析

In [ ]:
import openai
openai.api_key = my_api

In [ ]:
def ask_question(text, question):
    """Asks a question about the given text and returns the answer using OpenAI API."""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Based on the following Chinese text, answer the question:\n\nText: {text}\n\nQuestion: {question}\n\nAnswer:"}
        ],
        max_tokens=200,
        temperature=0.5,
    )
    answer = response.choices[0].message['content'].strip()
    return answer

In [ ]:
sorted_content_by_len = sorted(df_tg_zs['content'], key=len)
# pick top 100 long text from mblog contents.
chinese_text = ''.join(sorted_content_by_len[:100])
question = "请总结上述文本内容"
answer = ask_question(chinese_text, question)
print("\nAnswer to the question:")
print("------------------------")
print(answer)


Answer to the question:
------------------------
上述文本内容主要是关于陈丽君的一系列评论，主要指责她偷歌，即未经授权使用他人的歌曲。评论者对她的行为表示了强烈的不满和批评，有些评论者甚至对她的粉丝也进行了批评。同时，也有评论者对她的素质和道德进行了质疑。


In [ ]:
question = "上述文本是否含有侮辱性词语，如果有，请罗列出来"
answer = ask_question(chinese_text, question)
print("\nAnswer to the question:")
print("------------------------")
print(answer)


Answer to the question:
------------------------
是的，上述文本中包含了一些侮辱性词语，如“偷歌”、“恶心”、“明抢不要脸”、“偷歌真j”、“偷歌实证”、“素质感人”、“真有脸”、“厚颜无耻”、“假唱戏 真偷歌”、“黄腔最下流恶心”、“偷歌惯犯”、“出来道歉”、“什么都偷你也是无敌了”、“觉得自己糊就能偷是吧”、“实力没有倒是挺爱偷的十八线糊币网红粉逗我乐”、“自己


随便选几个关键词统计一下频次吧


In [ ]:
# function that counts the frequency of occurence of certain keywords/num of repost
def keyword_count(kw,df):
  cnt = 0
  repost_cnt = 0
  for index,row in df.iterrows():
    if kw in row['content']:
      cnt+=1
      repost_cnt+=row['repost_cnt']
  return [cnt,repost_cnt]

In [ ]:
abuse_words = ['文盲','侵权','卖姬','嘴硬','偷歌贼','倒反天罡','造谣']
print(f'在周深相关认证用户所发布的{df_tg_zs["微博id"].nunique()}条微博中：')
for word in abuse_words:
  print(f'关键词 {word}, 在{keyword_count(word, df_tg_zs)[0]}条微博中被提及,总传播次数为{keyword_count(word, df_tg_zs)[1]}次')

在周深相关认证用户所发布的590条微博中：
关键词 文盲, 在27条微博中被提及,总传播次数为359次
关键词 侵权, 在11条微博中被提及,总传播次数为9748次
关键词 卖姬, 在7条微博中被提及,总传播次数为10316次
关键词 嘴硬, 在30条微博中被提及,总传播次数为11437次
关键词 偷歌贼, 在12条微博中被提及,总传播次数为10300次
关键词 倒反天罡, 在6条微博中被提及,总传播次数为132次
关键词 造谣, 在23条微博中被提及,总传播次数为11049次


## 陈丽君回复周深粉丝 话题 小分析~~


一位名为 Stella_天晴 的微博用户于8月8日8点56分发表如下微博：


陈丽君女士搞的两次热搜相同点和不同点分析：
8月5日ct#陈丽君回应周深粉丝#.
8月7日ct#陈丽君回复周深粉丝#.
相同点👇🏻
1️⃣空降热搜时间：均为当天23点左右
2️⃣话题主持人：均为娱乐日爆社
3️⃣话题置顶：均为圈内爆爷
4️⃣发布媒体：4家相同媒体
5️⃣热搜位置：均为文娱磅高位，热搜磅低位
不同点👇🏻
1️⃣8月7日新增当事人陈丽君女士【因为第一次没关联上当事人所以又重新来一次吗？】
2️⃣8月7日有两家蓝v被去掉【可能太贵了？】
两次热搜仅一字之差，我还以为我进入循环了
看看话题阅读量和原创人数，米姐你不被蹭谁被蹭[泪]

补充：陈丽君粉丝一直在声称陈某身份为《体制内》，《党员》，我再补充一点陈某目前还是一个《公众人物》，但这三重身份并不是陈某粉丝以高高在上的姿态威胁网/暴普通人民群众的的保护伞，陈某刻意带素人两上热搜高位引导其粉丝网/暴他人的行为并不能与其身份相匹配，组织相信你给你身份是希望你能够更好的为人民群众服务，而不是让你粉丝作威作福欺负普通人民群众的，望了解@陈丽君_点绛唇

看看到底是不是如这位用户所说，是由陈老师方在买热搜。判断方法：如果是由陈老师或其相关方买热搜，那么炒热话题的就是陈老师的粉丝（职业粉丝）；反之则为周深方。

导入一下爬取的该话题数据

In [ ]:
df_res = pd.read_csv('./topics/20240808_2151_陈丽君回应周深粉丝话题相关微博.csv')
df_rep = pd.read_csv('./topics/20240808_2151_陈丽君回复周深粉丝话题相关微博.csv')
## res - response, rep - reply

让我们来看看圈内爆爷的微博记录：

以下为圈内爆爷关于陈老师回应/回复的两条微博

5064116207095894,圈内爆爷,#陈丽君回应周深粉丝#陈丽君回应了周深粉丝 关于自己对《光亮》歌词的唱腔质疑结果。 ,2024-08-05 21:00:53,发布于 河北,3005,7707,231437

5064873719370717,圈内爆爷,陈丽君本人回复了周深粉丝。#陈丽君回复周深粉丝# ,2024-08-07 23:10:58,发布于 河北,483,1630,108152

In [ ]:
# data type conversion
df_res['微博发布时间'] = pd.to_datetime(df_res['微博发布时间'])
df_rep['微博发布时间'] = pd.to_datetime(df_rep['微博发布时间'])

In [ ]:
rs_1 = pd.Timestamp(year=2024, month=8, day=5, hour=21, minute=0, second=53)
rs_2 = pd.Timestamp(year=2024, month=8, day=7, hour=23, minute=10, second=58)
df_res.fillna('',inplace=True)
df_rep.fillna('',inplace=True)

#### 前期数据整理完毕，进行分析

让我们先准备一点关键词

In [ ]:
clj = ['陈丽君','君君', '橙粒俊','老婆','俊小狗']  # 虽然你们叫的很肉麻，但是没办法嘛，周深老师反正不大像是老婆的样子
zs = ['周深','深深','啾星','反深代词','叻'] # 嗯。。反正是我总结出来的 from lots of names

In [ ]:
## 为了防止再漏掉一些，我们再加上之前 偷歌 话题的参与用户吧, 陈老师的粉丝大概率是不会转发带这种tag的微博的，除非是黑。
zs_fans = list(df_tg['微博发布者id'].unique())
## 至于陈老师的粉丝么，小弟不才，平时还是有搜集一点的，大概。。。几千个吧。
clj_fans = []
with open('./userinfo/clj_fans.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        columns = line.split(":")
        if len(columns) > 0:
            clj_fans.append(columns[0])
clj_fans = [int(x) for x in set(clj_fans) if x.isdigit()]



写个function来assign粉籍

In [ ]:
def fan_of_clj_or_zs(row):
    if row['微博发布者id'] in zs_fans:
        # return '周深粉 或 陈丽君黑'
        return '周深粉'
    elif row['微博发布者id'] in clj_fans:
        return '陈丽君粉'
    else:
        return ''

# Apply the function to the DataFrames
df_res['is_clj_or_zs'] = df_res.apply(fan_of_clj_or_zs, axis=1)
df_rep['is_clj_or_zs'] = df_rep.apply(fan_of_clj_or_zs, axis=1)

def is_zs_or_clj(df):
    def check_x(row):
        if any(substring in row['微博简介'] for substring in clj) or \
           any(substring in row['微博发布来源'] for substring in clj) or \
           any(substring in row['认证理由'] for substring in clj):
            return '陈丽君粉'
        elif any(substring in row['微博简介'] for substring in zs) or \
             any(substring in row['微博发布来源'] for substring in zs) or \
             any(substring in row['认证理由'] for substring in zs) or \
             any(substring in row['微博发布者'] for substring in zs):
            return '周深粉'
        else:
            return '吃瓜路人 或 营销号'

    df['is_zs_or_clj'] = df.apply(check_x, axis=1)
    return df

def fill_empty_rows(df):
    df = is_zs_or_clj(df)

    df.loc[df['is_clj_or_zs'] == '', 'is_clj_or_zs'] = df.loc[df['is_clj_or_zs'] == '', 'is_zs_or_clj']

    df.drop(columns=['is_zs_or_clj'], inplace=True)

    return df


In [ ]:
df_res = fill_empty_rows(df_res)
df_rep = fill_empty_rows(df_rep)

In [ ]:
df_rep.rename(columns={'is_clj_or_zs':'粉籍'},inplace=True)
df_res.rename(columns={'is_clj_or_zs':'粉籍'},inplace=True)

圈内爆爷营销之前 即早于8/5 21:00，谁在发布关于 陈丽君回应周深粉丝 相关的微博


In [ ]:
df_res_bf = df_res[df_res['微博发布时间']<rs_1].sort_values('微博发布时间')

圈内爆爷营销之后 即晚于8/5 21:00，谁在发布关于 陈丽君回应周深粉丝 相关的微博

In [ ]:
df_res_af = df_res[df_res['微博发布时间']>rs_1].sort_values('微博发布时间')

根据粉籍：周深粉/陈丽君粉/路人或者营销号 做一些统计

In [ ]:
def df_grouped_pct(df_raw):
    df = df_raw.groupby('粉籍')\
        .agg(总转发数=('repost_cnt', 'sum'),
             总点赞数=('like_cnt', 'sum'),
             参与人数=('微博发布者id', 'nunique'))\
        .reset_index()

    total_repost = df['总转发数'].sum()
    total_like = df['总点赞数'].sum()
    total_participants = df['参与人数'].sum()

    df['转发数百分比'] = round(df['总转发数'] / total_repost * 100,2)
    df['点赞数百分比'] = round(df['总点赞数'] / total_like * 100,2)
    df['参与人数百分比'] = round(df['参与人数'] / total_participants * 100,2)

    return df


圈内爆爷主持话题前 # 陈丽君回应周深粉丝前 两方粉丝/营销号或路人 所占百分比

In [ ]:
df_grouped_pct(df_res_bf)

圈内爆爷主持话题后 # 陈丽君回应周深粉丝前 两方粉丝/营销号或路人 所占百分比

In [ ]:
df_grouped_pct(df_res_af)

In [ ]:
df_rep_bf = df_rep[df_rep['微博发布时间']<rs_2].sort_values('微博发布时间')
df_rep_af = df_rep[df_rep['微博发布时间']>rs_2].sort_values('微博发布时间')

In [ ]:
df_grouped_pct(df_rep_bf)

In [ ]:
df_grouped_pct(df_rep_af)

看一下 陈老师的粉丝里有哪些是在 周深方挑起的话题里有发表微博的。

In [ ]:
def get_suspicious_clj_fan(df):
  res = df[df['粉籍']=='陈丽君粉'].groupby('微博发布者')\
  .agg(info = ('认证理由','first'),
       mblog_cnt = ('微博id','nunique'),
       first_mblog = ('微博发布时间','min'),
       last_mblog = ('微博发布时间','max'),
       like_cnt = ('like_cnt','sum'))\
  .sort_values('mblog_cnt',ascending=False).reset_index()
  return res

In [ ]:
get_suspicious_clj_fan(df_res)

In [ ]:
get_suspicious_clj_fan(df_rep)

In [ ]:
poster_1 = list(get_suspicious_clj_fan(df_res)['微博发布者'].unique())
poster_2 = list(get_suspicious_clj_fan(df_rep)['微博发布者'].unique())

In [ ]:
np.intersect1d(poster_1, poster_2)